## 네이버 아티클 selenium 

In [14]:
category = {
    100 : "정치",
    101 : "경제",
    102 : "사회",
    103 : "생활/문화",
    104 : "세계",
    105 : "IT/과학",
}
category, date, page = 100, 20200826, 1

url = "https://news.naver.com/main/list.nhn?\
mode=LSD&mid=sec&sid1={}&date={}&page={}".format(category, date, page)

In [15]:
print(url)

https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1=100&date=20200826&page=1


In [16]:
driver = webdriver.Chrome()
driver.get(url)

In [17]:
articles = driver.find_elements_by_css_selector(".list_body > ul > li")
len(articles)

20

In [19]:
title = articles[0].find_element_by_css_selector("dt:not(.photo) > a").text #photo가 없는 dt만
link = articles[0].find_element_by_css_selector("dt:not(.photo) > a")\
.get_attribute("href")
title, print(link)

https://news.naver.com/main/read.nhn?mode=LSD&mid=sec&sid1=100&oid=421&aid=0004834408


('모두발언하는 정세균 총리', None)

In [20]:
response = requests.get(link)
dom = BeautifulSoup(response.content, "html.parser")

In [22]:
content = dom.select_one("#articleBodyContents").text\
.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].split("▶")[0]
content

'(서울=뉴스1) 임세영 기자 = 정세균 국무총리가 26일 오전 서울 세종로 정부서울청사에서 열린 신종 코로나바이러스 감염증(코로나19) 대응 중앙재난안전대책본부회의 겸 제8호 태풍 바비(BAVI) 대처상황 점검회의에서 모두발언을 하고 있다. 이날 정 총리는 의사들을 향해 “환자 생명을 담보로 한 집단행동은 국민이 용납하지 않을것” 이라며 “즉시 진료 업무에 복귀하라”고 촉구했다. 2020.8.26/뉴스1seiyu@news1.kr'

In [23]:
def get_articles(category, date, page):
    
    url = "https://news.naver.com/main/list.nhn?mode=LSD&mid=sec&sid1={}&date={}&page={}"\
    .format(category, date, page)
    options = webdriver.ChromeOptions()
    options.add_argument("headless")
    driver = webdriver.Chrome(options=options)
    driver.get(url)
    articles = driver.find_elements_by_css_selector(".list_body > ul > li")
    
    datas = []
    for article in articles:
        title = article.find_element_by_css_selector("dt:not(.photo) > a").text
        link = article.find_element_by_css_selector("dt:not(.photo) > a").get_attribute("href")
        response = requests.get(link)
        dom = BeautifulSoup(response.content, "html.parser")
        content = dom.select_one("#articleBodyContents").text.replace("\n", "").replace("\'", "").replace("- ", "").split("{}")[1].strip()
        datas.append({
            "title": title,
            "link": link,
            "content": content,
            "category": category,
        })
    driver.quit()
    
    return pd.DataFrame(datas)

In [26]:
categories = {
    100: "정치",
    101: "경제"
}

In [27]:
dfs = []
date = 20191121

for category in categories:
    print(categories[category], end=" ") # categories[category] => value 반환
    for page in range(1, 6):
        print(page, end=" ")
        df = get_articles(category, date, page)
        dfs.append(df)
    print()
    
articles_df = pd.concat(dfs, ignore_index=True) # 리스트에다가 쭉 df 집어넣고 concat하면 하나로 합쳐짐
articles_df.tail()

정치 1 2 3 4 5 
경제 1 2 3 4 5 


,title,link,content,category
195,"남다른감자탕 ㈜보하라, 2019 독서경영 우수 직장으로 선정",https://news.naver.com/main/read.nhn?mode=LSD&...,건강감자탕브랜드 남다른감자탕을 운영하는 ㈜보하라는 문화체육관광부에서 주최하는 201...,101
196,"기아차 신차 출시 한 달 앞서 공개, 왜?",https://news.naver.com/main/read.nhn?mode=LSD&...,【 앵커멘트 】 기아차가 오늘 K5 신차를 공개했습니다. 보통 출시 직전에 오...,101
197,광역전철 운행도 감축 … 시민 불편 가중,https://news.naver.com/main/read.nhn?mode=LSD&...,철도노조 총파업 이틀째 / 1호선 등 평시 대비 82% 운행 / 출퇴근길 승객 몰려...,101
198,"캐세이퍼시픽항공, 동남아 6개 도시 비즈니스석 특가",https://news.naver.com/main/read.nhn?mode=LSD&...,캐세이퍼시픽항공이 연말연시 여행을 계획하고 있는 이들을 위해 동남아 6개 인기 도시...,101
199,코엑스-독일농업협회 글로벌 푸드테크전시회 개최키로,https://news.naver.com/main/read.nhn?mode=LSD&...,(서울=연합뉴스) 코엑스와 독일농업협회가 21일 오후 서울 강남구 삼성동 코엑스 그...,101


In [28]:
articles_df.to_csv("article.csv", index=False, encoding="utf-8-sig")

In [29]:
driver.quit()